In [30]:
import pyodbc
import pandas as pd
import numpy as np
import os

# Conexión a Azure SQL
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'
azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

# Conexión a SQL Server LOCAL
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'  
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# Nombre del archivo SQL que contiene la consulta
SQL_FILE = "../DATAEX.FACT_SALES.sql"  # Cambia esto por tu ruta de archivo

# Nombre de la tabla en SQL Server Local
NEW_TABLE_NAME = "DATAEX.FACT_SALES"

def leer_consulta_desde_archivo(archivo):
    """Lee la consulta SQL desde un archivo"""
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Error al leer el archivo SQL: {e}")
        raise

try:
    # 1. Leer la consulta desde el archivo SQL
    print(f"Leyendo consulta desde el archivo {SQL_FILE}...")
    SQL_QUERY = leer_consulta_desde_archivo(SQL_FILE)
    
    # 2. Conectar a Azure SQL
    print(f"Conectando a Azure SQL...")
    conn_azure = pyodbc.connect(azure_conn_str)
    
    # 3. Ejecutar la consulta en Azure SQL
    print(f"Ejecutando consulta en Azure SQL...")
    df = pd.read_sql(SQL_QUERY, conn_azure)

    if df.empty:
        print(f"La consulta no devolvió resultados. No se creará la tabla en SQL Server Local.")
    else:
        print(f"Datos extraídos: {df.shape[0]} filas")

        # 4. Limpieza y preparación de datos
        df = df.fillna(0)
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)
        
        # 5. Conectar a SQL Server Local
        print(f"Conectando a SQL Server Local...")
        conn_local = pyodbc.connect(local_conn_str)
        
        with conn_local.cursor() as cursor:
            # 6. Eliminar tabla existente si es necesario
            drop_table_sql = f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}"
            cursor.execute(drop_table_sql)
            conn_local.commit()
            print(f"Tabla existente eliminada (si existía)")

            # 7. Crear nueva tabla
            create_table_sql = f"""
            CREATE TABLE {NEW_TABLE_NAME} (
                {', '.join([
                    f'[{col}] FLOAT' if df[col].dtype == np.float32 
                    else f'[{col}] INT' if df[col].dtype == np.int32 
                    else f'[{col}] NVARCHAR(255)' for col in df.columns
                ])}
            );
            """
            cursor.execute(create_table_sql)
            conn_local.commit()
            print(f"Tabla {NEW_TABLE_NAME} creada correctamente")

            # 8. Insertar datos
            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"
            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()
            print(f"{df.shape[0]} filas insertadas en {NEW_TABLE_NAME}")

except Exception as e:
    print(f"Error durante el proceso: {e}")
    if 'conn_local' in locals():
        conn_local.rollback()

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("\n¡Proceso completado!")

Leyendo consulta desde el archivo ../DATAEX.FACT_SALES.sql...
Conectando a Azure SQL...
Ejecutando consulta en Azure SQL...


C:\Users\josit\AppData\Local\Temp\ipykernel_28840\2236020593.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL_QUERY, conn_azure)


Datos extraídos: 58049 filas
Conectando a SQL Server Local...
Tabla existente eliminada (si existía)
Tabla DATAEX.FACT_SALES creada correctamente
58049 filas insertadas en DATAEX.FACT_SALES

¡Proceso completado!


In [ ]:
import pyodbc
import os
import re

def ejecutar_script_sql_en_sqlserver():
    try:
        # Configuración de conexión
        SERVER = 'localhost'
        DATABASE = 'dwh_case1'
        DRIVER = '{ODBC Driver 17 for SQL Server}'
        conn_str = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes'
        
        # Archivo SQL a procesar
        archivo_sql = '../claves/FACT/VALORES_NULOS_FACTS.sql'
        
        if not os.path.exists(archivo_sql):
            raise FileNotFoundError(f"El archivo {archivo_sql} no existe")

        print("Conectando a SQL Server...")
        with pyodbc.connect(conn_str) as conn:
            conn.autocommit = False
            cursor = conn.cursor()
            
            # Leer y procesar el archivo SQL correctamente
            with open(archivo_sql, 'r', encoding='utf-8') as file:
                contenido = file.read()
                
                # 1. Eliminar comentarios (-- y /* */)
                contenido = re.sub(r'--.*?\n', '', contenido)  # Comentarios de línea
                contenido = re.sub(r'/\*.*?\*/', '', contenido, flags=re.DOTALL)  # Bloques de comentarios
                
                # 2. Separar sentencias individuales correctamente
                sentencias = []
                buffer = ""
                for linea in contenido.split('\n'):
                    linea = linea.strip()
                    if not linea:
                        continue
                    
                    buffer += " " + linea if buffer else linea
                    
                    # Verificar si tenemos una sentencia completa
                    if ';' in buffer:
                        partes = buffer.split(';')
                        for parte in partes[:-1]:
                            parte = parte.strip()
                            if parte:
                                sentencias.append(parte)
                        buffer = partes[-1].strip()
                
                # Añadir cualquier sentencia restante
                if buffer.strip():
                    sentencias.append(buffer.strip())
            
            # Filtrar y ejecutar solo sentencias ALTER COLUMN válidas
            for i, sentencia in enumerate(sentencias, 1):
                try:
                    # Validar que sea una sentencia ALTER TABLE ALTER COLUMN
                    if not re.match(r'^\s*ALTER\s+TABLE\s+\w+\.\w+\s+ALTER\s+COLUMN', sentencia, re.IGNORECASE):
                        print(f"\n[{i}] Omitiendo (no es ALTER COLUMN válido): {sentencia[:100]}...")
                        continue
                        
                    print(f"\n[{i}] Ejecutando: {sentencia[:100]}...")
                    
                    cursor.execute(sentencia)
                    conn.commit()
                    print("  ✓ Éxito")
                    
                except pyodbc.Error as e:
                    print(f"\n❌ Error en la sentencia {i}:")
                    print(f"Código SQL: {e.args[0]}")
                    print(f"Mensaje: {e.args[1]}")
                    print(f"Sentencia completa:\n{sentencia}")
                    
                    # Manejo especial para errores de conversión de NULL a NOT NULL
                    if "cannot alter column" in str(e.args[1]).lower() and "because one or more objects access this column" in str(e.args[1]).lower():
                        print("\n⚠️ Posible solución: Eliminar constraints dependientes antes de modificar la columna")
                    
                    continuar = input("\n¿Continuar con las siguientes sentencias? (s/n): ").lower()
                    if continuar != 's':
                        conn.rollback()
                        print("Proceso cancelado por el usuario")
                        return
            
            print("\n¡Proceso completado con éxito!")
            
    except Exception as e:
        print(f"\n⚠️ Error general: {str(e)}")
        if 'conn' in locals():
            conn.rollback()
    finally:
        if 'conn' in locals():
            conn.close()

# Ejecutar la función
ejecutar_script_sql_en_sqlserver()

Conectando a SQL Server...

[1] Ejecutando: ALTER TABLE DATAEX.DIM_LUGAR ALTER TABLE DATAEX.DIM_PRODUCTO ALTER TABLE DATAEX.FACT_SALES ALTER TAB...

❌ Error en la sentencia 1:
Código de error: 42000
Mensaje: [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de la palabra clave 'TABLE'. (156) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de la palabra clave 'TABLE'. (156); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de la palabra clave 'TABLE'. (156); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de la palabra clave 'TABLE'. (156); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de la palabra clave 'TABLE'. (156); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de la palabra clave 'TABLE'. (156); [42000] [Microsoft][ODBC Driver

In [16]:
import pyodbc
import os

def ejecutar_script_sql_en_sqlserver():
    try:
        # Configuración de conexión (ajusta según tu entorno)
        SERVER = 'localhost'
        DATABASE = 'dwh_case1'
        DRIVER = '{ODBC Driver 17 for SQL Server}'
        
        conn_str = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes'
        
        print("Conectando a SQL Server...")
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        archivo_sql = '../claves/FACT/VALORES_NULOS_FACTS.sql'
        if not os.path.exists(archivo_sql):
            raise FileNotFoundError(f"El archivo {archivo_sql} no existe")

        with open(archivo_sql, 'r', encoding='utf-8') as file:
            sql_script = file.read()
        
        # Solución: Reemplazar el \n por un espacio directamente
        sentencias = [s.strip().replace('\n', ' ') for s in sql_script.split('GO') if s.strip() and not s.strip().startswith('--')]
        
        for i, sentencia in enumerate(sentencias, 1):
            try:
                if sentencia:
                    # Versión corregida sin \n en el f-string
                    print(f"\nEjecutando sentencia {i}: {sentencia[:50]}...")
                    cursor.execute(sentencia)
                    conn.commit()
                    print("✓ Éxito")
            except pyodbc.Error as e:
                print(f"\n❌ Error en la sentencia {i}:")
                print(f"Código de error: {e.args[0]}")
                print(f"Mensaje: {e.args[1]}")
                print(f"Sentencia problemática:\n{sentencia[:200]}...")
                
                continuar = input("\n¿Continuar con las siguientes sentencias? (s/n): ").lower()
                if continuar != 's':
                    break
        
        print("\n¡Proceso completado!")
        
    except Exception as e:
        print(f"\n⚠️ Error general: {str(e)}")
        if 'conn' in locals():
            conn.rollback()
    finally:
        if 'conn' in locals():
            conn.close()

ejecutar_script_sql_en_sqlserver()

Conectando a SQL Server...

Ejecutando sentencia 1: _POSTAL_LIMPIO nvarchar(255) NOT NULL;  ALTER TABL...

❌ Error en la sentencia 1:
Código de error: 42000
Mensaje: [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Sintaxis incorrecta cerca de '255'. (102) (SQLExecDirectW)
Sentencia problemática:
_POSTAL_LIMPIO nvarchar(255) NOT NULL;  ALTER TABLE DATAEX.DIM_CLIENTE ALTER COLUMN Edad int NOT NULL;  ALTER TABLE DATAEX.DIM_CLIENTE ALTER COLUMN GENERO nvarchar(255) NOT NULL;  ALTER TABLE DATAEX.D...

¡Proceso completado!


In [34]:
import pyodbc
import os

def aplicar_constraints():
    # Configuración de conexión
    SERVER = 'localhost'
    DATABASE = 'dwh_case1'
    DRIVER = '{ODBC Driver 17 for SQL Server}'
    conn_str = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes'
    
    # Nombre del archivo SQL
    SQL_FILE = "../claves/FACT/PKS_FACT_SALES.sql"
    
    try:
        # Verificar que el archivo existe
        if not os.path.exists(SQL_FILE):
            raise FileNotFoundError(f"El archivo {SQL_FILE} no existe")
        
        # Leer el contenido del archivo SQL
        with open(SQL_FILE, 'r', encoding='utf-8') as file:
            sql_script = file.read()
        
        # Conectar a SQL Server
        print("Conectando a SQL Server...")
        with pyodbc.connect(conn_str) as conn:
            conn.autocommit = False  # Importante para el manejo de transacciones
            cursor = conn.cursor()
            
            # Verificar si la tabla existe
            cursor.execute("""
                SELECT COUNT(*) 
                FROM INFORMATION_SCHEMA.TABLES 
                WHERE TABLE_SCHEMA = 'DATAEX' 
                AND TABLE_NAME = 'FACT_SALES'
            """)
            if cursor.fetchone()[0] == 0:
                raise Exception("La tabla DATAEX.FACT_SALES no existe")
            
            # Separar las sentencias SQL
            sentencias = [s.strip() for s in sql_script.split(';') if s.strip()]
            
            for i, sentencia in enumerate(sentencias, 1):
                try:
                    if not sentencia:
                        continue
                        
                    # Mostrar la sentencia (sin saltos de línea para evitar el error)
                    print(f"\n[{i}] Ejecutando: {sentencia[:50].replace(chr(10), ' ')}...")
                    
                    # Verificación especial para FOREIGN KEYS
                    if "FOREIGN KEY" in sentencia.upper():
                        tabla_ref = sentencia.upper().split("REFERENCES")[1].split("(")[0].strip()
                        print(f"  → Verificando integridad referencial con {tabla_ref}...")
                        
                        # Ejecutar con WITH CHECK para forzar validación
                        sentencia = sentencia.replace(");", ") WITH CHECK;")
                    
                    cursor.execute(sentencia)
                    conn.commit()
                    print("  ✓ Éxito")
                    
                except pyodbc.Error as e:
                    print(f"\n❌ Error en la sentencia {i}:")
                    print(f"Código SQL: {e.args[0]}")
                    print(f"Mensaje: {e.args[1]}")
                    
                    # Opción para intentar con NOCHECK si falla
                    if "FOREIGN KEY" in sentencia.upper():
                        print("\nIntentando con WITH NOCHECK...")
                        try:
                            sentencia_nocheck = sentencia.replace(");", ") WITH NOCHECK;")
                            cursor.execute(sentencia_nocheck)
                            conn.commit()
                            print("  ✓ Aplicada con NOCHECK (no se validaron datos existentes)")
                        except pyodbc.Error as e2:
                            print(f"  ✖ Error con NOCHECK: {e2.args[1]}")
                            conn.rollback()
                    
                    continuar = input("\n¿Continuar con las siguientes constraints? (s/n): ").lower()
                    if continuar != 's':
                        conn.rollback()
                        print("Proceso cancelado por el usuario")
                        return
            
            # Verificación final de constraints aplicadas
            print("\nResumen de constraints aplicadas:")
            cursor.execute("""
                SELECT 
                    CONSTRAINT_NAME, 
                    CONSTRAINT_TYPE 
                FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS 
                WHERE TABLE_SCHEMA = 'DATAEX' 
                AND TABLE_NAME = 'FACT_SALES'
            """)
            for row in cursor:
                print(f"  - {row.CONSTRAINT_NAME}: {row.CONSTRAINT_TYPE}")
            
            print("\n¡Proceso completado!")
    
    except Exception as e:
        print(f"\n⚠️ Error general: {str(e)}")
        if 'conn' in locals():
            conn.rollback()

# Ejecutar la función
aplicar_constraints()

Conectando a SQL Server...

[1] Ejecutando: -- Establecer CODE como clave primaria ALTER TABLE...
  ✓ Éxito

[2] Ejecutando: -- Establecer Customer_ID como clave foránea que r...
  → Verificando integridad referencial con DATAEX.DIM_CLIENTE...
  ✓ Éxito

[3] Ejecutando: -- Establecer Id_Producto como clave foránea que r...
  → Verificando integridad referencial con DATAEX.DIM_PRODUCTO...
  ✓ Éxito

[4] Ejecutando: -- Establecer Sales_Date como clave foránea que re...
  → Verificando integridad referencial con DATAEX.DIM_TIEMPO...
  ✓ Éxito

[5] Ejecutando: -- Establecer TIENDA_ID como clave foránea que ref...
  → Verificando integridad referencial con DATAEX.DIM_LUGAR...
  ✓ Éxito

Resumen de constraints aplicadas:
  - PK_DATAEXFACT_SALES: PRIMARY KEY
  - FK_Customer_ID: FOREIGN KEY
  - FK_Id_Producto: FOREIGN KEY
  - FK_DATE: FOREIGN KEY
  - FK_TIENDA_ID: FOREIGN KEY

¡Proceso completado!


In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import os
import time

# Configuración de conexiones
CONFIG = {
    'azure': {
        'server': 'uaxmathfis.database.windows.net',
        'database': 'usecases',
        'driver': '{ODBC Driver 17 for SQL Server}',
        'auth': 'Authentication=ActiveDirectoryInteractive'
    },
    'local': {
        'server': 'localhost',
        'database': 'dwh_case1',
        'driver': '{ODBC Driver 17 for SQL Server}',
        'auth': 'Trusted_Connection=yes;TrustServerCertificate=yes'
    }
}

# Lista de tablas a procesar (nombre de tabla: archivo SQL)
TABLAS_A_IMPORTAR = {
    "DATAEX.FACT_SALES": "../consultas_dimensional/DATAEX.FACT_SALES.sql",
    "DATAEX.DIM_CLIENTE": "../consultas_dimensional/DATAEX.DIM_CLIENTE.sql",
    "DATAEX.DIM_PRODUCTO": "../consultas_dimensional/DATAEX.DIM_PRODUCTO.sql",
    "DATAEX.DIM_LUGAR": "../consultas_dimensional/DATAEX.DIM_LUGAR.sql",
    "DATAEX.DIM_TIEMPO": "../consultas_dimensional/DATAEX.DIM_TIEMPO.sql",
}

def get_connection_string(connection_type):
    """Genera la cadena de conexión según el tipo"""
    cfg = CONFIG[connection_type]
    return f"DRIVER={cfg['driver']};SERVER={cfg['server']};DATABASE={cfg['database']};{cfg['auth']}"

def test_connection(connection_type):
    """Prueba la conexión a SQL Server"""
    try:
        with pyodbc.connect(get_connection_string(connection_type), timeout=10) as conn:
            print(f"✅ Conexión exitosa a {connection_type}")
            return True
    except Exception as e:
        print(f"❌ Error conectando a {connection_type}: {e}")
        return False

def leer_consulta_desde_archivo(archivo):
    """Lee la consulta SQL desde un archivo"""
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"❌ Error al leer el archivo SQL: {e}")
        return None

def crear_tabla_local(cursor, table_name, df):
    """Crea una tabla local e inserta los datos"""
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    cursor.connection.commit()
    print(f"  - Tabla eliminada (si existía)")

    create_table_sql = f"""
    CREATE TABLE {table_name} (
        {', '.join([f'[{col}] FLOAT' if df[col].dtype == np.float32 
                    else f'[{col}] INT' if df[col].dtype == np.int32 
                    else f'[{col}] NVARCHAR(255)' for col in df.columns])}
    );
    """
    cursor.execute(create_table_sql)
    cursor.connection.commit()
    print(f"  - Tabla {table_name} creada")

    placeholders = ', '.join(['?' for _ in df.columns])
    insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
    cursor.fast_executemany = True
    cursor.executemany(insert_sql, df.values.tolist())
    cursor.connection.commit()
    print(f"  - {df.shape[0]} filas insertadas")

def procesar_tabla(table_name, sql_file):
    """Procesa una tabla individual"""
    print(f"\n{'='*50}")
    print(f"📌 Procesando tabla: {table_name}")
    print(f"📄 Archivo SQL: {sql_file}")
    print(f"{'='*50}")

    sql_query = leer_consulta_desde_archivo(sql_file)
    if not sql_query:
        return False

    try:
        # Conectar a Azure y obtener datos
        with pyodbc.connect(get_connection_string('azure')) as conn_azure:
            print("  - ⏳ Extrayendo datos de Azure SQL...")
            df = pd.read_sql(sql_query, conn_azure)

            if df.empty:
                print("  - ⚠️ No hay datos para importar")
                return False

            # Limpieza de datos
            df.fillna(0, inplace=True)
            for col in df.select_dtypes(include=['float64']).columns:
                df[col] = df[col].astype(np.float32)
            for col in df.select_dtypes(include=['int64']).columns:
                df[col] = df[col].astype(np.int32)

            # Conectar a SQL local e importar datos
            with pyodbc.connect(get_connection_string('local')) as conn_local:
                with conn_local.cursor() as cursor:
                    crear_tabla_local(cursor, table_name, df)

        return True

    except pyodbc.Error as e:
        print(f"❌ Error de conexión con SQL Server: {str(e)}")
        return False
    except Exception as e:
        print(f"❌ Error procesando tabla {table_name}: {str(e)}")
        return False

def main():
    print(f"\n{'*'*50}")
    print(f"🚀 INICIANDO IMPORTACIÓN DE {len(TABLAS_A_IMPORTAR)} TABLAS")
    print(f"{'*'*50}")

    # Probar conexiones antes de ejecutar
    if not test_connection('azure') or not test_connection('local'):
        print("⛔ No se puede continuar. Revisa la conexión a SQL Server.")
        return

    exitosas = 0
    for table_name, sql_file in TABLAS_A_IMPORTAR.items():
        if procesar_tabla(table_name, sql_file):
            exitosas += 1

    print(f"\n{'*'*50}")
    print(f"✅ RESUMEN DE EJECUCIÓN")
    print(f"✔️ Tablas procesadas: {len(TABLAS_A_IMPORTAR)}")
    print(f"✔️ Importaciones exitosas: {exitosas}")
    print(f"❌ Errores: {len(TABLAS_A_IMPORTAR) - exitosas}")
    print(f"{'*'*50}")

if __name__ == "__main__":
    main()



**************************************************
INICIANDO IMPORTACIÓN DE 5 TABLAS
**************************************************

Procesando tabla: DATAEX.FACT_SALES
Archivo SQL: ../consultas_dimensional/DATAEX.FACT_SALES.sql
  - Error procesando tabla DATAEX.FACT_SALES: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [64].  (64) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (64)')

Procesando tabla: DATAEX.DIM_CLIENTE
Archivo SQL: ../consultas_dimensional/DATAEX.DIM_CLIENTE.sql
  - Error procesando tabla DATAEX.DIM_CLI

In [ ]:
import pyodbc
import os

def aplicar_constraints():
    # Configuración de conexión
    SERVER = 'localhost'
    DATABASE = 'dwh_case1'
    DRIVER = '{ODBC Driver 17 for SQL Server}'
    conn_str = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes'
    
    # Estructura de archivos SQL a procesar en orden
    SQL_FILES = {
        'dimensiones': [
            "../claves/DIM/PKS_DIM_CLIENTE.sql",
            "../claves/DIM/PKS_DIM_PRODUCTO.sql",
            "../claves/DIM/PKS_DIM_TIEMPO.sql",
            "../claves/DIM/PKS_DIM_LUGAR.sql"
        ],
        'hechos': [
            "../claves/FACT/PKS_FACT_SALES.sql",
            "../claves/FACT/FKS_FACT_SALES.sql"
        ]
    }
    
    try:
        # Conectar a SQL Server
        print("Conectando a SQL Server...")
        with pyodbc.connect(conn_str) as conn:
            conn.autocommit = False
            cursor = conn.cursor()
            
            # Procesar en dos fases: primero dimensiones, luego hechos
            for fase in ['dimensiones', 'hechos']:
                print(f"\n{'='*50}")
                print(f"PROCESANDO {fase.upper()}")
                print(f"{'='*50}")
                
                for sql_file in SQL_FILES[fase]:
                    try:
                        # Verificar que el archivo existe
                        if not os.path.exists(sql_file):
                            print(f"⚠️ Archivo no encontrado: {sql_file}")
                            continue
                        
                        # Leer el contenido del archivo SQL
                        with open(sql_file, 'r', encoding='utf-8') as file:
                            sql_script = file.read()
                        
                        # Extraer nombre de tabla del nombre de archivo
                        table_name = os.path.basename(sql_file).split('.')[1].split('_')[-1]
                        full_table_name = f"DATAEX.{table_name}"
                        
                        # Verificar si la tabla existe
                        cursor.execute(f"""
                            SELECT COUNT(*) 
                            FROM INFORMATION_SCHEMA.TABLES 
                            WHERE TABLE_SCHEMA = 'DATAEX' 
                            AND TABLE_NAME = '{table_name}'
                        """)
                        if cursor.fetchone()[0] == 0:
                            print(f"⚠️ Tabla {full_table_name} no existe. Saltando...")
                            continue
                        
                        # Separar las sentencias SQL
                        sentencias = [s.strip() for s in sql_script.split(';') if s.strip()]
                        
                        for i, sentencia in enumerate(sentencias, 1):
                            try:
                                if not sentencia:
                                    continue
                                    
                                # Mostrar la sentencia (sin saltos de línea)
                                print(f"\n[{i}] Ejecutando en {full_table_name}: {sentencia[:50].replace(chr(10), ' ')}...")
                                
                                # Verificación especial para FOREIGN KEYS
                                if "FOREIGN KEY" in sentencia.upper():
                                    tabla_ref = sentencia.upper().split("REFERENCES")[1].split("(")[0].strip()
                                    print(f"  → Verificando integridad referencial con {tabla_ref}...")
                                    
                                    # Ejecutar con WITH CHECK para forzar validación
                                    sentencia = sentencia.replace(");", ") WITH CHECK;")
                                
                                cursor.execute(sentencia)
                                conn.commit()
                                print("  ✓ Éxito")
                                
                            except pyodbc.Error as e:
                                print(f"\n❌ Error en la sentencia {i}:")
                                print(f"Código SQL: {e.args[0]}")
                                print(f"Mensaje: {e.args[1]}")
                                
                                # Opción para intentar con NOCHECK si falla
                                if "FOREIGN KEY" in sentencia.upper():
                                    print("\nIntentando con WITH NOCHECK...")
                                    try:
                                        sentencia_nocheck = sentencia.replace(");", ") WITH NOCHECK;")
                                        cursor.execute(sentencia_nocheck)
                                        conn.commit()
                                        print("  ✓ Aplicada con NOCHECK (no se validaron datos existentes)")
                                    except pyodbc.Error as e2:
                                        print(f"  ✖ Error con NOCHECK: {e2.args[1]}")
                                        conn.rollback()
                                
                                continuar = input("\n¿Continuar con las siguientes constraints? (s/n): ").lower()
                                if continuar != 's':
                                    conn.rollback()
                                    print("Proceso cancelado por el usuario")
                                    return
                    
                    except Exception as e:
                        print(f"\n⚠️ Error procesando archivo {sql_file}: {str(e)}")
                        continue
            
            # Verificación final de todas las constraints aplicadas
            print("\nResumen final de constraints aplicadas:")
            for fase in ['dimensiones', 'hechos']:
                print(f"\n{fase.upper()}:")
                for sql_file in SQL_FILES[fase]:
                    table_name = os.path.basename(sql_file).split('.')[1].split('_')[-1]
                    full_table_name = f"DATAEX.{table_name}"
                    
                    cursor.execute(f"""
                        SELECT 
                            CONSTRAINT_NAME, 
                            CONSTRAINT_TYPE 
                        FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS 
                        WHERE TABLE_SCHEMA = 'DATAEX' 
                        AND TABLE_NAME = '{table_name}'
                    """)
                    constraints = cursor.fetchall()
                    
                    if constraints:
                        print(f"\n{full_table_name}:")
                        for row in constraints:
                            print(f"  - {row.CONSTRAINT_NAME}: {row.CONSTRAINT_TYPE}")
                    else:
                        print(f"\n{full_table_name}: No se aplicaron constraints")
            
            print("\n¡Proceso completado para todas las tablas!")
    
    except Exception as e:
        print(f"\n⚠️ Error general: {str(e)}")
        if 'conn' in locals():
            conn.rollback()

# Ejecutar la función
aplicar_constraints()